In [2]:
import bezier
from tkinter import *
import numpy as np
from PIL import ImageGrab

In [4]:
print(ord('\u4E00')-ord('\u9FFF'))

-20991


In [3]:
#define global canvas and stuff
canvasSize = 800

# Create object and size
root = Tk()
root.geometry( "800x850" )

#add label for draw char, label, text
charLabel = Label(root, text = "Char: ")
charLabel.pack()

#handle user typing in a char
entryText = StringVar()
charEntry = Entry(root, width = 10, justify="center", textvariable=entryText)
entryText.set(drawChar) #default char
charEntry.bind("<KeyPress>", onTextClear) #user is about to finish typing a char
charEntry.bind("<KeyRelease>", onTextChange) #user entered a char
charEntry.pack()

#button for processing the button
processBtn = Button(root, text ="Gen. Strokes", command = saveCanvasAsImage)
processBtn.pack()

#setup canvas
canvas = Canvas(root, bg="white", height=canvasSize, width=canvasSize) #our drawing board
canvas.create_text(400, 400, text=drawChar, font=('Microsoft YaHei','500'), fill='#555555')
clickEventClear(0) #default arg of 0 as the event arg isn't used
canvas.pack()

#reset the lists before we re-run the tkinter app
currChar = []
currStroke = []

canvas.bind("<Button-1>", clickEventPoint) #setup the left click event, add a control point
root.bind("<Key>", clickEventClear) #setup the left click event, clear the canvas
canvas.bind("<Double-Button-1>", clickEventStroke) #setup the right click event, finalize stroke

root.mainloop() #run the app

NameError: name 'drawChar' is not defined